<a href="https://colab.research.google.com/github/trotsak/text/blob/main/text_ml_pre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Название проекта: Классификация текстов для интернет-магазина «Викишоп»**.

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75.

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели.
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

In [1]:
!pip install -q catboost contractions

In [2]:
!python -m spacy download en_core_web_md -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 21.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:
# работа с операционной системой
import os

## Импорт библиотек для анализа и визуализации данных

# работа с данными в формате таблиц
import pandas as pd

# работа с многомерными массивами
import numpy as np

# Импортируем функцию sqrt (квадратный корень) из модуля math
from math import sqrt

# Импортируем функцию autocorrelation_plot из библиотеки pandas.plotting
from pandas.plotting import autocorrelation_plot

# визуализация данных
import matplotlib.pyplot as plt

# импорт функции display для отображения датафреймов и других объектов в Jupyter Notebook
from IPython.display import display

# Импорт модуля re для работы с регулярными выражениями
import re

# импорт конфигурационных параметров для настройки отображения графиков
from matplotlib import rcParams, rcParamsDefault

# расширенные возможности визуализации
import seaborn as sns

# Импортируем модуль time для работы со временем
import time

## Импорт библиотек для статистического анализа

# статистические функции
from scipy import stats as st

# Импортируем функцию adfuller из модуля statsmodels.tsa.stattools
from statsmodels.tsa.stattools import adfuller

# Импорт функции для выполнения теста KPSS (Kwiatkowski-Phillips-Schmidt-Shin)
# для проверки стационарности временного ряда
from statsmodels.tsa.stattools import kpss

# Импортируем функции для визуализации автокорреляции и частичной автокорреляции
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# импорт библиотеки для статистического анализа
import scipy.stats as stats

# Импорт функции seasonal_decompose из библиотеки statsmodels.
from statsmodels.tsa.seasonal import seasonal_decompose

## Импорт библиотек для машинного обучения

# Импортируем ColumnTransformer для предобработки данных по столбцам
from sklearn.compose import ColumnTransformer

# Импортируем Pipeline для создания конвейера обработки данных и обучения модели
from sklearn.pipeline import Pipeline

# Импортируем функцию set_config из библиотеки scikit-learn
from sklearn import set_config


# общая библиотека машинного обучения
import sklearn

# разделение данных и оценка моделей
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, RandomizedSearchCV

# кодирование категориальных переменных и стандартизация
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler

# Импортируем класс LinearRegression из библиотеки sklearn (подмодуля linear_model)
# from sklearn.linear_model import LinearRegression

# Импортируем класс RandomForestRegressor из библиотеки sklearn (подмодуля ensemble)
# from sklearn.ensemble import RandomForestRegressor

# Импортируем KNNImputer из библиотеки sklearn.impute
# from sklearn.impute import KNNImputer

# Импорт модели LightGBM
# from lightgbm import LGBMRegressor

# Импортируем lightgbm как lgb
import lightgbm as lgb

# Импорт функции mean_squared_error из библиотеки scikit-learn
from sklearn.metrics import mean_squared_error, make_scorer

## Импорт библиотеки для обработки предупреждений

# управление предупреждениями
import warnings

# игнорировать предупреждения (если нужно)
# warnings.filterwarnings('ignore')

import optuna
import re
import string
import requests
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')
wnl = WordNetLemmatizer()


stopwords = set(nltk_stopwords.words('english'))

from sklearn.metrics import f1_score


from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression


import spacy

nlp = spacy.load("en_core_web_sm")

from tqdm import tqdm
tqdm.pandas()

# Создание общего прогресс-бара для apply
tqdm.pandas(desc="Общий прогресс")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
class Color:

    """
    Класс для хранения цветовых кодов для форматирования текстов в терминале.
    """

    PURPLE = '\033[95m'      # Фиолетовый цвет
    CYAN = '\033[96m'        # Бирюзовый цвет
    DARK_CYAN = '\033[36m'   # Темно-бирюзовый цвет
    BLUE = '\033[94m'        # Синий цвет
    GREEN = '\033[92m'       # Зеленый цвет
    YELLOW = '\033[93m'      # Желтый цвет
    RED = '\033[91m'         # Красный цвет
    BOLD = '\033[1m'         # Жирный текст
    UNDERLINE = '\033[4m'    # Подчеркнутый текст
    END = '\033[0m'          # Сброс формата текста


In [5]:
# определение констант
RANDOM_STATE = 42
TEST_SIZE = 0.10
CV_COUNTS=5

In [6]:
# системные настройки
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

### Настроим параметры отображения графиков в Matplotlib для лучшей визуализации и качества изображения.

In [7]:
# установка стиля графиков на основе библиотеки Seaborn
sns.set_style('whitegrid')

# Включаем отображение объектов scikit-learn в виде диаграммы
set_config(display='diagram')

# установка формата изображения SVG для обеспечения более четкого и качественного изображение графиков.
%config InlineBackend.figure_format = 'svg'

# масштабный фактор, который будет использоваться для изменения параметра dpi.
factor = 0.8

# извлечение значения по умолчанию для точек на дюйм (dpi) из настроек Matplotlib.
default_dpi = rcParamsDefault['figure.dpi']

# установка разрешения (dpi) для всех фигур путём умножения значения dpi на масштабный фактор.
rcParams['figure.dpi'] = default_dpi*factor

# включение отображения графиков в Jupyter
%matplotlib inline

# установка размера диаграмм
rcParams['figure.figsize'] = [12.0, 6.0]

## Подготовка

### Загрузим данные из csv-файла в датафрейм.

In [8]:
## считывание данных из csv-файлов в датафреймы

# назначение путей к файлам
file_paths = {
    'toxic_comments': '/datasets/ toxic_comments.csv'
}

# словарь для хранения загруженных данных
dataframes = {}

# проход по всем файлам
for name, path in file_paths.items():
    try:
        if os.path.exists(path):
            dataframes[name] = pd.read_csv(path)
            print(f'Файл {path} загружен из локального пути.')
        else:
            url = f'https://code.s3.yandex.net/datasets/{name}.csv'
            dataframes[name] = pd.read_csv(url)
            print(f'Файл {path} загружен из URL.')
    except Exception as e:
        print(f'Не удалось загрузить {path}: {e}')

# присваивание загруженным датафреймам отдельных переменных
comments_data = dataframes['toxic_comments']

Файл /datasets/ toxic_comments.csv загружен из URL.


## Анализ

### Изучим общую информацию о полученном датафрейме
Создадим функцию `data_info` для вывода общей информации по датафрейму.

In [9]:
# создание функции для вывода общей информации по датафрейму
def data_info(data, dataframe_name):
    """
    Отображает общую информацию о переданном датафрейме.

    Функция выполняет следующие операции:
    1. Отображение первых нескольких строк датафрейма.
    2. Вывод общей информации о датафрейме, включая типы данных и количество ненулевых значений.
    3. Отображение статистического описания числовых столбцов.
    4. Подсчет и вывод количества пропущенных значений в каждом столбце.
    5. Вывод количества явных дубликатов в датафрейме.
    6. Отображение списка названий столбцов в датафрейме.
    7. Вывод уникальных значений для столбцов с типом данных 'object'.
    8. Вывод числа уникальных значений для каждого столбца.
    9. Вывод числа дублей для каждого столбца.

    Параметры:
    ----------
    data : pandas.DataFrame
        Датафрейм, для которого необходимо вывести информацию.
    dataframe_name : str
        Имя датафрейма (для отображения в выводе).
    """

    # отображение первых несколько строк датафрейма
    print(Color.BOLD + f"Первые строки датафрейма {dataframe_name}:\n" + Color.END)
    display(data.head())
    print()

    # вывод информацию о датафрейме, включая типы данных и количество ненулевых значений
    print(Color.BOLD + f"Общая информация о датафрейме {dataframe_name}:\n" + Color.END)
    data.info()
    print()

    # отображение статистического описания числовых столбцов датафрейма
    print(Color.BOLD + f"Статистическое описание числовых столбцов датафрейма {dataframe_name}:\n" + Color.END)
    display(data.describe())
    print()

    # отображение количества пропущенных значений в каждом столбце
    print(Color.BOLD + f"Количества пропущенных значений в каждом столбце датафрейма {dataframe_name}:\n" + Color.END)
    display(data.isna().sum())
    print()

    # вывод количества явных дубликатов в датафрейме
    print(f'Количество явных дубликатов в датафрейме: {Color.RED}{data.duplicated().sum()}{Color.END}.')
    print()

    # отображение списка названий столбцов в датафрейме
    print(Color.BOLD + f"Cписок названий столбцов в датафрейме {dataframe_name}:\n" + Color.END)
    display(data.columns.tolist())

    # отображение всех уникальных значений и их количества в столбцах типа 'object'
    for i in data.columns:
        if data[i].dtype == 'object':
            unique_values = data[i].unique()
            num_unique_values = len(unique_values)
            num_duplicates = data[i].duplicated().sum()
            print(f'В столбце {Color.BOLD}\'{i}\'{Color.END} содержится {num_unique_values} уникальных значений: \
            {Color.BOLD}{unique_values}{Color.END}')
            print(f'Число дублей в столбце {Color.BOLD}\'{i}\'{Color.END}: {Color.RED}{num_duplicates}{Color.END}')

#### Общая информация о датафреме `taxi_data`:

In [10]:
# получение общей информации по датафрейму с помощью функции data_info
data_info(comments_data, 'comments_data')

Первые строки датафрейма comments_data:



,Unnamed: 0,text,toxic
0,0,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",0
1,1,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",0
2,2,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",0
3,3,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of """"types of accidents"""" -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know.\n\nThere appears to be a backlog on articles for review so I guess there may be a delay until a reviewer turns up. It's listed in the relevant form eg Wikipedia:Good_article_nominations#Transport """,0
4,4,"You, sir, are my hero. Any chance you remember what page that's on?",0



Общая информация о датафрейме comments_data:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB

Статистическое описание числовых столбцов датафрейма comments_data:



,Unnamed: 0,toxic
count,159292.000000,159292.000000
mean,79725.697242,0.101612
std,46028.837471,0.302139
min,0.000000,0.000000
25%,39872.750000,0.000000
50%,79721.500000,0.000000
75%,119573.250000,0.000000
max,159450.000000,1.000000



Количества пропущенных значений в каждом столбце датафрейма comments_data:



,0
Unnamed: 0,0
text,0
toxic,0



Количество явных дубликатов в датафрейме: 0.

Cписок названий столбцов в датафрейме comments_data:



['Unnamed: 0', 'text', 'toxic']

В столбце 'text' содержится 159292 уникальных значений:             ["Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"
 "D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC)"
 "Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info."
 ...
 'Spitzer \n\nUmm, theres no actual article for prostitution ring.  - Crunch Captain.'
 'And it looks like it was actually you who put on the speedy to have the first version deleted now that I look at it.'
 '"\nAnd ... I really don\'t think you understand.  I came here and my idea was bad right away.  W

**Выводы:**

В датафрейме `taxi_data` содержится 4416 строк и 2 колонки с численными и временными типами данных. Индекс отсортирован в монотонном порядке.

Согласно документации колонки содержат следующую информацию:

- `datetime` - время заказов такси,
- `num_orders` - количество заказов такси.

Подписи колонок соответствуют хорошему стилю написания, пропущенные значения и аномалии отсутствуют, даты расположены в хронологическом порядке, присутствуют дубли, но в данной ситуации это нормально, удалять не будем, количество заказов такси может повторяться.

За 1-часовой период: в среднем 84 заказа, типичная нагрузка в 75% случаев составляет до 107 заказов, изменчивость умеренная - стандартное отклонение 45,  максимальное число заказов значительно отличается от значения среднего и медианы - 462, что указывает на наличие всплесков спроса; минимальное число заказов - 0.

In [11]:
comments_data = comments_data.drop('Unnamed: 0', axis=1)
display(comments_data.head())
comments_data.info()

,text,toxic
0,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",0
1,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",0
2,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",0
3,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of """"types of accidents"""" -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know.\n\nThere appears to be a backlog on articles for review so I guess there may be a delay until a reviewer turns up. It's listed in the relevant form eg Wikipedia:Good_article_nominations#Transport """,0
4,"You, sir, are my hero. Any chance you remember what page that's on?",0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [12]:
# Оптимизируем тип данных признака 'toxic' (int64 переведем в формат uint8):
comments_data['toxic'] = comments_data['toxic'].astype('uint8')

In [13]:
comments_data.groupby('toxic')['text'].count()

,text
toxic,
0,143106
1,16186


In [14]:
# from tqdm import tqdm
# def lemmatize_text(text):

#     # Удаляем символы, не относящиеся к русскому алфавиту
#     sub_text = re.sub(r'[^a-zA-Z ]', ' ', text)
#     join_text = " ".join(sub_text.split())

#     doc = nlp(join_text)

#     lemmatized_tokens = []


#     # Добавляем прогресс-бар с помощью tqdm
#     for token in doc:
#         lemmatized_tokens.append(token.lemma_)

#     # Удаляем лишние пробелы и объединяем лемматизированные токены
#     lem_text = " ".join(lemmatized_tokens)

#     return lem_text

In [15]:
# lemmatize_text("The striped bats are hanging on their feet for best")

In [16]:
nlp.pipe_names

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

In [17]:
disabled_pipes = [ "parser",  "ner"]
nlp = spacy.load('en_core_web_sm', disable=disabled_pipes)

In [18]:
# lemm_texts = []

# for doc in tqdm(nlp.pipe(comments_data['text'].values, disable = ['ner', 'parser']),
#                 total=comments_data.shape[0],
#                 desc="Обработка текста"
#                 ):
#         lemm_text = " ".join([i.lemma_ for i in doc])
#         lemm_texts.append(lemm_text)

In [19]:
# # import spacy

# # # Загрузка модели Spacy для английского языка
# # nlp = spacy.load("en_core_web_sm")

# # Исходное предложение
# sentence = "The striped bats are hanging on their feet for best"

# # Применение модели Spacy
# doc = nlp(sentence)

# # Лемматизированный текст
# lemm_text = " ".join([token.lemma_ for token in doc])

# print("Исходный текст:", sentence)
# print("Лемматизированный текст:", lemm_text)

In [20]:
# # Проверяем первые N строк
# for i, lemm_text in enumerate(lemm_texts[:5]):
#     print(f"Текст {i + 1}: {comments_data['text'].iloc[i]}")
#     print(f"Лемматизированный: {lemm_text}")
#     print("-" * 50)

In [21]:
# Результат
# comments_data['lemm_texts'] = lemm_texts

In [22]:
# comments_data.head()

In [23]:
# comments_data['lem_text'] = comments_data['text'].progress_apply(lemmatize_text)
# print(comments_data['lem_text'].head())

In [24]:
# Загружаем более мощную модель spaCy
nlp = spacy.load('en_core_web_md', disable=["ner", "parser", "textcat"])

In [25]:
import contractions

# Функция очистки текста
def clean_text(text):
    text = contractions.fix(text)  # Разворачивает сокращения
    text = text.lower()  # Приводим к нижнему регистру
    text = re.sub(r'<.*?>', '', text)  # Удаляем HTML-теги
    text = re.sub(r'\([^)]*\)', '', text)  # Удаляем текст в скобках
    text = re.sub(r'\d{1,2}:\d{2}', '', text)  # Убираем время (формат 21:51)
    text = re.sub(r'\d{4}', '', text)  # Удаляем года (например, 2016)
    text = re.sub(r'\b(january|february|march|april|may|june|july|august|september|october|november|december)\b \d{1,2},? \d{4}', '', text)  # Убираем даты
    text = re.sub(r'[^a-z\s]', '', text)  # Оставляем только буквы и пробелы
    text = re.sub(r'\s+', ' ', text).strip()  # Убираем лишние пробелы
    return text

In [26]:
# # Функция для очистки текста
# def clean_text(text):
#     text = text.lower()  # Приводим к нижнему регистру
#     text = re.sub(r'\([^)]*\)', '', text)  # Удаляем скобки и их содержимое
#     text = re.sub(r'\d{1,2}:\d{2}', '', text)  # Удаляем время (например, 21:51)
#     text = re.sub(r'\d{4}', '', text)  # Удаляем года (например, 2016)
#     text = re.sub(r'\s+', ' ', text).strip()  # Убираем лишние пробелы
#     return text

In [27]:
# Применяем очистку к каждому комментарию
comments_data['clean_text'] = comments_data['text'].apply(clean_text)

In [28]:
extra_stopwords = {"d'aww", "daww", "hey", "sir"}  # Расширяем список стоп-слов
lemm_texts = []
for doc in tqdm(nlp.pipe(comments_data['clean_text'].tolist()), total=len(comments_data), desc="Лемматизация"):

    lemm_text = " ".join([
    token.lemma_
    for token in doc
    if not token.is_stop and            # Исключаем стоп-слова
       not token.is_punct and           # Исключаем пунктуацию
       token.lemma_ not in extra_stopwords and  # Исключаем дополнительные стоп-слова
       len(token.text) > 2              # Убираем слишком короткие слова
])



    # lemm_text = " ".join([token.lemma_ for token in doc if not token.is_stop and not token.is_punct])
    # lemm_text = " ".join([token.lemma_ for token in doc if not token.is_stop and not token.is_punct not in extra_stopwords and len(token.text) > 2])
    # lemm_text = " ".join([token.lemma_ for token in doc if token.text not in extra_stopwords])
    lemm_texts.append(lemm_text)

comments_data["lemm_text"] = lemm_texts  # Добавляем в DataFrame

Лемматизация: 100%|██████████| 159292/159292 [13:42<00:00, 193.74it/s]


In [29]:
comments_data.head()

,text,toxic,clean_text,lemm_text
0,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",0,explanation why the edits made under my username hardcore metallica fan were reverted they were not vandalisms just closure on some gas after i voted at new york dolls fac and please do not remove the template from the talk page since i am retired now,explanation edit username hardcore metallica fan revert vandalism closure gas vote new york dolls fac remove template talk page retire
1,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",0,daww he matches this background colour i am seemingly stuck with thanks january,match background colour seemingly stuck thank january
2,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",0,hey man i am really not trying to edit war it is just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page he seems to care more about the formatting than the actual info,man try edit war guy constantly remove relevant information talk edit instead talk page care formatting actual info
3,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of """"types of accidents"""" -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know.\n\nThere appears to be a backlog on articles for review so I guess there may be a delay until a reviewer turns up. It's listed in the relevant form eg Wikipedia:Good_article_nominations#Transport """,0,more i cannot make any real suggestions on improvement i wondered if the section statistics should be later on or a subsection of types of accidents i think the references may need tidying so that they are all in the exact same format ie date format etc i can do that later on if noone else does first if you have any preferences for formatting style on references or want to do it yourself please let me know there appears to be a backlog on articles for review so i guess there may be a delay until a reviewer turns up it is listed in the relevant form eg wikipediagoodarticlenominationstransport,real suggestion improvement wonder section statistic later subsection type accident think reference need tidy exact format date format etc later preference format style reference want let know appear backlog article review guess delay reviewer turn list relevant form wikipediagoodarticlenominationstransport
4,"You, sir, are my hero. Any chance you remember what page that's on?",0,you sir are my hero any chance you remember what page that is on,hero chance remember page


In [30]:
# Проверяем первые N строк
for i, lemm_text in enumerate(lemm_texts[:5]):
    print(f"Текст {i + 1}: {comments_data['text'].iloc[i]}")
    print(f"Лемматизированный: {lemm_text}")
    print("-" * 50)

Текст 1: Explanation
Why the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27
Лемматизированный: explanation edit username hardcore metallica fan revert vandalism closure gas vote new york dolls fac remove template talk page retire
--------------------------------------------------
Текст 2: D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC)
Лемматизированный: match background colour seemingly stuck thank january
--------------------------------------------------
Текст 3: Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.
Лемматизированный: man try edit 

## Добавим новые признаки

In [31]:
# # Добавить длину текста как признак

# comments_data["text_length"] = comments_data["text"].apply(len)
# comments_data["word_count"] = comments_data["text"].apply(lambda x: len(x.split()))

In [32]:
# # Количество восклицательных и вопросительных знаков

# comments_data["excl_marks"] = comments_data["text"].apply(lambda x: x.count("!"))
# comments_data["quest_marks"] = comments_data["text"].apply(lambda x: x.count("?"))

In [33]:
# # Процент заглавных букв

# comments_data["caps_ratio"] = comments_data["text"].apply(lambda x: sum(1 for c in x if c.isupper()) / len(x) if len(x) > 0 else 0)

In [34]:
# # Наличие токсичных слов

# toxic_words = {"stupid", "idiot", "hate", "dumb", "bitch", "fuck", "suck", "moron"}
# comments_data["toxic_words_count"] = comments_data["lemm_text"].apply(lambda x: sum(1 for word in x.split() if word in toxic_words))

In [35]:
# # Количество ссылок и упоминаний
# import re

# comments_data["num_links"] = comments_data["text"].apply(lambda x: len(re.findall(r'http[s]?://', x)))
# comments_data["num_mentions"] = comments_data["text"].apply(lambda x: len(re.findall(r'@\w+', x)))

 преобразовать текст в числовой формат для модели

 Векторизация текста (TF-IDF)

In [36]:
from tqdm import tqdm
tqdm.pandas()
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=150_000, ngram_range=(1,2))
# X = vectorizer.fit_transform(comments_data["lemm_text"])
X = vectorizer.fit_transform(tqdm(comments_data["lemm_text"], desc="Векторизация TF-IDF"))
y = comments_data["toxic"]

Векторизация TF-IDF: 100%|██████████| 159292/159292 [00:11<00:00, 14338.44it/s]


In [37]:
# from tqdm import tqdm
# tqdm.pandas()  # Подключаем tqdm к Pandas

# vectorizer = TfidfVectorizer(max_features=100_000, ngram_range=(1,2))

# # Прогресс-бар при обработке текстов
# X = vectorizer.fit_transform(tqdm(comments_data["lemm_text"], desc="Векторизация TF-IDF"))


In [38]:
# from sklearn.feature_extraction.text import TfidfVectorizer

# vectorizer = TfidfVectorizer(max_features=5000)  # Ограничиваем до 5000 слов
# X = vectorizer.fit_transform(comments_data["lemm_text"])
# y = comments_data["toxic"]

In [39]:
# comments_data.info()

In [40]:
# # Создание матрицы из остальных численных признаков
# additional_features = comments_data[[
#     "text_length", "word_count", "excl_marks", "quest_marks",
#     "caps_ratio", "toxic_words_count", "num_links", "num_mentions"
# ]].values

In [41]:
# # Масштабируем дополнительные признаки
# scaler = StandardScaler()
# scaled_additional_features = scaler.fit_transform(
#     comments_data[["text_length", "word_count", "excl_marks", "quest_marks",
#                    "caps_ratio", "toxic_words_count", "num_links", "num_mentions"]].values
# )

In [42]:
# from scipy.sparse import hstack

# Объединение TF-IDF с дополнительными признаками
# X = hstack([X_tfidf, additional_features])

In [43]:
# Целевая переменная
# y = comments_data["toxic"]

In [44]:
# import random

# # Строка X[0]
# row = X[0]

# # Печать случайных 5 ненулевых элементов
# sample_indices = random.sample(range(len(row.data)), k=min(5, len(row.data)))  # Случайные индексы
# for idx in sample_indices:
#     print(f"Feature {row.indices[idx]}: {row.data[idx]}")

In [45]:
# X


In [46]:
# y.sample(5)

In [47]:
# print(X.shape)

In [48]:
# print(vectorizer.get_feature_names_out())

Разделение на обучающую и тестовую выборку

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

Обучение модели логистической регрессии

In [50]:
model = LogisticRegression(class_weight="balanced", max_iter=100, random_state=RANDOM_STATE)
model.fit(X_train, y_train)

LogisticRegression(class_weight='balanced', random_state=42)

In [51]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'C': [0.1, 1, 10],  # Регуляризация
    # 'penalty': ['l1', 'l2'],
    'solver': ['liblinear', 'lbfgs']
}

grid_search = GridSearchCV(LogisticRegression(class_weight="balanced", max_iter=500, random_state=42),
                           param_grid, scoring='f1', cv=5, n_jobs=-1)
grid_search.fit(X, y)

print(f"Лучшие параметры: {grid_search.best_params_}")
print(f"Средний F1-score: {grid_search.best_score_:.4f}")

/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
15 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py", line 1193, in fit
    solver = _check_solver

Лучшие параметры: {'C': 10, 'penalty': 'l2', 'solver': 'liblinear'}
Средний F1-score: 0.7726


In [52]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import GridSearchCV

# param_grid = {
#     'C': [0.01, 0.1, 1, 10],  # Чем меньше C, тем сильнее регуляризация
#     'penalty': ['l1', 'l2'],  # L1 - Lasso, L2 - Ridge
#     'solver': ['liblinear']  # liblinear поддерживает L1
# }

# grid_search = GridSearchCV(LogisticRegression(class_weight="balanced", max_iter=500, random_state=42),
#                            param_grid, scoring='f1', cv=5, n_jobs=-1)
# grid_search.fit(X_train, y_train)

# best_model = grid_search.best_estimator_

# # Оценка лучшей модели
# y_pred_best = best_model.predict(X_test)
# f1_best = f1_score(y_test, y_pred_best)
# print(f"F1-score лучшей модели: {f1_best:.4f}")

Оценка качества модели

In [53]:
# from sklearn.metrics import f1_score, accuracy_score

# y_pred = model.predict(X_test)

# f1 = f1_score(y_test, y_pred)
# acc = accuracy_score(y_test, y_pred)

# print(f"F1-score: {f1:.4f}")
# print(f"Accuracy: {acc:.4f}")

1. Улучшить векторизацию текста
Попробуем TF-IDF с биграммами и триграммами, чтобы учесть соседние слова:

In [54]:
# from tqdm import tqdm
# import numpy as np

# vectorizer = TfidfVectorizer(max_features=70_000, ngram_range=(1,2))

# batch_size = 10_000  # Размер батча
# lemm_texts = comments_data["lemm_text"].tolist()
# X_parts = []

# for i in tqdm(range(0, len(lemm_texts), batch_size), desc="Обработка TF-IDF"):
#     X_part = vectorizer.fit_transform(lemm_texts[i:i+batch_size])
#     X_parts.append(X_part)

# X = np.vstack(X_parts)  # Объединяем обратно


### CatBoost

In [55]:
!pip install catboost -q

In [56]:
# Обучим CatBoost с кросс-валидацией (StratifiedKFold для дисбаланса классов):
from sklearn.model_selection import cross_val_score, StratifiedKFold
from catboost import CatBoostClassifier

# Модель CatBoost
cat_model = CatBoostClassifier(iterations=200, depth=4, learning_rate=0.1,
                               loss_function='Logloss',
                               verbose=100,
                               random_state=42,
                               od_type="Iter",
                               od_wait=50,
                               thread_count=-1)


# Кросс-валидация (StratifiedKFold для дисбаланса классов)

cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
cv_scores = cross_val_score(cat_model, X, y, cv=cv, scoring="f1")

print(f"Средний F1-score CatBoost: {cv_scores.mean():.4f}")
print(f"Разброс значений: {cv_scores}")


0:	learn: 0.5940681	total: 2.75s	remaining: 9m 6s
100:	learn: 0.1799754	total: 2m 23s	remaining: 2m 20s
199:	learn: 0.1589509	total: 4m 38s	remaining: 0us
0:	learn: 0.5959025	total: 1.47s	remaining: 4m 51s
100:	learn: 0.1804374	total: 2m 18s	remaining: 2m 15s
199:	learn: 0.1588406	total: 4m 32s	remaining: 0us
0:	learn: 0.5919778	total: 1.47s	remaining: 4m 52s
100:	learn: 0.1795783	total: 2m 23s	remaining: 2m 20s
199:	learn: 0.1586686	total: 4m 34s	remaining: 0us
Средний F1-score CatBoost: 0.6736
Разброс значений: [0.66911678 0.68258659 0.66896962]


In [57]:
# cat_model.fit(X_train, y_train)

# # Оценка на тесте
# y_pred_cat = cat_model.predict(X_test)
# print(f"F1-score CatBoost: {f1_score(y_test, y_pred_cat):.4f}")


### BERT

In [58]:
!pip install transformers torch datasets -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 862.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [59]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_scheduler
from sklearn.model_selection import train_test_split


In [60]:
# Загружаем предобученный токенизатор BERT

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Токенизация
def tokenize_texts(texts):
    return tokenizer(texts, padding=True, truncation=True, max_length=512, return_tensors="pt")

# Разбиваем данные
X_train, X_test, y_train, y_test = train_test_split(comments_data["lemm_text"], comments_data["toxic"], test_size=0.2, stratify=comments_data["toxic"], random_state=42)

# Токенизируем
train_encodings = tokenize_texts(X_train.tolist())
test_encodings = tokenize_texts(X_test.tolist())


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [61]:
#  Создаём Dataset для PyTorch

class ToxicDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels.iloc[idx])
        return item

# Создаём PyTorch dataset
train_dataset = ToxicDataset(train_encodings, y_train)
test_dataset = ToxicDataset(test_encodings, y_test)

# DataLoader для обучения
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)


In [62]:
# Загружаем предобученный BERT для классификации
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)  # Два класса: токсичный / нетоксичный
model.to("cuda" if torch.cuda.is_available() else "cpu")


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [63]:
# Настраиваем оптимизатор и лосс-функцию
optimizer = AdamW(model.parameters(), lr=5e-5)
loss_fn = torch.nn.CrossEntropyLoss()


/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
#  Обучаем BERT
device = "cuda" if torch.cuda.is_available() else "cpu"
model.train()

for epoch in range(3):  # 3 эпохи
    total_loss = 0
    for batch in train_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    print(f"Эпоха {epoch + 1}, Средний loss: {total_loss / len(train_loader):.4f}")


In [ ]:
# Оцениваем качество (F1-score)
from sklearn.metrics import f1_score

model.eval()
predictions, true_labels = [], []

with torch.no_grad():
    for batch in test_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=-1).cpu().numpy()
        labels = batch["labels"].cpu().numpy()

        predictions.extend(preds)
        true_labels.extend(labels)

# Оцениваем F1-score
f1 = f1_score(true_labels, predictions)
print(f"F1-score BERT: {f1:.4f}")


## Объединяем TF-IDF с новыми признаками

In [ ]:
# import scipy.sparse as sp

# X_meta = comments_data[["text_length", "word_count", "excl_marks", "quest_marks", "caps_ratio", "toxic_words_count", "num_links", "num_mentions"]]
# X_meta = sp.csr_matrix(X_meta)  # Преобразуем в sparse-формат


In [ ]:
# print(f"Размер TF-IDF: {X.shape}")  # Должно быть (159292, N)
# print(f"Размер X_meta: {X_meta.shape}")  # Должно быть (159292, M)


In [ ]:
# # Проверим, есть ли пропущенные значения в comments_data
# print(comments_data.isnull().sum())  # Есть ли NaN в колонках?
# print(comments_data.shape)  # Должно быть (159292, ...)


In [ ]:
# comments_data["lemm_text"] = comments_data["lemm_text"].fillna("")


In [ ]:
# print(comments_data.index[:5])
# print(pd.DataFrame(X.toarray()).index[:5])  # Индексы TF-IDF
# print(X_meta.index[:5])  # Индексы доп. признаков


In [ ]:
# print(f"Размер TF-IDF: {X.shape}")
# print(f"Размер X_meta: {X_meta.shape}")


In [ ]:
# Проверим размеры данных перед объединением

In [ ]:
# X_combined = sp.hstack([X, X_meta])  # Объединяем с TF-IDF

In [ ]:
# print(comments_data.shape)  # Должно быть (159292, N)
# print(comments_data.isnull().sum())  # Проверяем NaN


In [ ]:
# print(comments_data[["text_length", "word_count", "excl_marks", "quest_marks",
                    #  "caps_ratio", "toxic_words_count", "num_links", "num_mentions"]].isnull().sum())


In [ ]:
# import scipy.sparse as sp

# X_meta = comments_data[["text_length", "word_count", "excl_marks", "quest_marks",
#                         "caps_ratio", "toxic_words_count", "num_links", "num_mentions"]]
# X_meta = sp.csr_matrix(X_meta)  # Преобразуем в sparse

# # Проверяем размеры
# print(f"Размер TF-IDF: {X.shape}")
# print(f"Размер X_meta: {X_meta.shape}")


In [ ]:
# X_combined = sp.hstack([X, X_meta])  # Объединяем с TF-IDF

In [ ]:
# import numpy as np

# # Check shapes
# print(f"Shape of X: {X.shape}")
# print(f"Length of y: {len(y)}")

# # Optionally check for mismatched indices or missing values
# if hasattr(X, 'index') and hasattr(y, 'index'):
#     print(f"Mismatched indices? {not np.array_equal(X.index, y.index)}")

# # Example adjust lengths (if you know how to align)
# min_length = min(len(X), len(y))
# X = X[:min_length]  # Truncate X
# y = y[:min_length]  # Truncate y

In [ ]:
# rows = sparse_matrix.shape[0]
# print(f"Number of rows in sparse matrix: {rows}")

# print(f"Length of y: {len(y)}")

In [ ]:
# model.fit(X_combined, y_train)